Imports & config

In [ ]:
# CELL 1 — Imports & config

import pandas as pd
import torch
from pathlib import Path
import json

DATA_DIR = Path("../data/processed")
OUT = DATA_DIR / "tensor_shards"
OUT.mkdir(exist_ok=True)

MAX_PREFIX_LEN = 20
PAD_ID = 0
UNK_ID = 1
SHARD_SIZE = 250_000

print("[05B] Output dir:", OUT.resolve())


Load manifest & splits